<a href="https://colab.research.google.com/github/AbdulSheffa/IRP/blob/main/FYP_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 7.89 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix


In [3]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.8 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha

In [15]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
from PIL import Image
from models.networks import ResnetGenerator, NLayerDiscriminator

# ------------------------------
# 1. Dataset with Self-Supervised Learning Masking
# ------------------------------
class RainDatasetSSL(Dataset):
    def __init__(self, dataset_name, input_dir, target_dir, transform=None, sample_size=None, output_dir=None, save_masked=False, epoch=0):
        self.dataset_name = dataset_name
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.output_dir = output_dir
        self.save_masked = save_masked  # Flag to control saving masked images
        self.epoch = epoch  # Current epoch for saving masked images with correct names

        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))

        if sample_size:
            indices = random.sample(range(len(self.input_images)), sample_size)
            self.input_images = [self.input_images[i] for i in indices]
            self.target_images = [self.target_images[i] for i in indices]

        if save_masked and output_dir:
            os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    def __len__(self):
        return len(self.input_images)  # ✅ Fix for DataLoader issue

    def mask_image(self, img_tensor, img_name):
        """Apply dynamic masking for self-supervised learning & save masked image if required."""
        _, h, w = img_tensor.shape
        mask_size = (random.randint(h // 4, h // 2), random.randint(w // 4, w // 2))  # Dynamic mask size
        x, y = random.randint(0, h - mask_size[0]), random.randint(0, w - mask_size[1])

        # Apply masking (set pixels to zero in the selected region)
        img_tensor[:, x:x + mask_size[0], y:y + mask_size[1]] = 0

        # Save masked image if enabled
        if self.save_masked and self.output_dir:
            save_path = os.path.join(self.output_dir, f"masked_{self.dataset_name}_epoch_{self.epoch}_de_rained_{img_name}")
            save_image(img_tensor, save_path)
            print(f"Saved masked image: {save_path}")

        return img_tensor

    def __getitem__(self, idx):
        input_img_path = os.path.join(self.input_dir, self.input_images[idx])
        target_img_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_img_path).convert("RGB")
        target_image = Image.open(target_img_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        # Apply masking to input image for SSL
        masked_image = self.mask_image(input_image.clone(), self.input_images[idx])

        return masked_image, input_image, target_image, self.input_images[idx]

# ------------------------------
# 2. Improved Self-Supervised DeRain CycleGAN Model
# ------------------------------
class DeRainCycleGANSSL:
    def __init__(self, input_nc, output_nc, ngf, ndf, device):
        self.device = device
        self.netG_A = ResnetGenerator(input_nc, output_nc, ngf).to(device)  # Rainy -> Clean
        self.netG_B = ResnetGenerator(output_nc, input_nc, ngf).to(device)  # Clean -> Rainy
        self.netD_A = NLayerDiscriminator(output_nc, ndf).to(device)  # Discriminator for Clean Images
        self.netD_B = NLayerDiscriminator(input_nc, ndf).to(device)  # Discriminator for Rainy Images

        self.criterionGAN = nn.MSELoss()
        self.criterionCycle = nn.L1Loss()

        # Add perceptual loss using VGG19
        self.vgg = models.vgg19(pretrained=True).features[:16].eval().to(device)
        for param in self.vgg.parameters():
            param.requires_grad = False
        self.criterionPerceptual = nn.MSELoss()

        # Optimizers
        self.optimizer_G = torch.optim.Adam(
            list(self.netG_A.parameters()) + list(self.netG_B.parameters()), lr=0.0001, betas=(0.5, 0.999)
        )
        self.optimizer_D = torch.optim.Adam(
            list(self.netD_A.parameters()) + list(self.netD_B.parameters()), lr=0.0001, betas=(0.5, 0.999)
        )

    def perceptual_loss(self, x, y):
        return self.criterionPerceptual(self.vgg(x), self.vgg(y))

    def forward(self, real_A, real_B):
        fake_B = self.netG_A(real_A)  # Rainy -> Clean
        rec_A = self.netG_B(fake_B)  # Clean -> Rainy
        fake_A = self.netG_B(real_B)  # Clean -> Rainy
        rec_B = self.netG_A(fake_A)  # Rainy -> Clean
        return fake_B, rec_A, fake_A, rec_B

    def backward_G(self, real_A, real_B, fake_B, rec_A, fake_A, rec_B):
        loss_G_A = self.criterionGAN(self.netD_A(fake_B), torch.ones_like(self.netD_A(fake_B)))
        loss_G_B = self.criterionGAN(self.netD_B(fake_A), torch.ones_like(self.netD_B(fake_A)))
        loss_cycle_A = self.criterionCycle(rec_A, real_A) * 10.0
        loss_cycle_B = self.criterionCycle(rec_B, real_B) * 10.0
        loss_perceptual = self.perceptual_loss(fake_B, real_B) * 0.1  # Small weight for perceptual loss

        loss_G = loss_G_A + loss_G_B + loss_cycle_A + loss_cycle_B + loss_perceptual
        loss_G.backward()
        return loss_G

    def backward_D(self, netD, real, fake):
        """ Compute discriminator loss and backpropagate """
        pred_real = netD(real)
        loss_D_real = self.criterionGAN(pred_real, torch.ones_like(pred_real))

        pred_fake = netD(fake.detach())  # Detach to avoid affecting generator
        loss_D_fake = self.criterionGAN(pred_fake, torch.zeros_like(pred_fake))

        loss_D = (loss_D_real + loss_D_fake) * 0.5
        loss_D.backward()
        return loss_D

    def optimize_parameters(self, real_A, real_B):
        fake_B, rec_A, fake_A, rec_B = self.forward(real_A, real_B)
        self.optimizer_G.zero_grad()
        loss_G = self.backward_G(real_A, real_B, fake_B, rec_A, fake_A, rec_B)
        self.optimizer_G.step()

        self.optimizer_D.zero_grad()
        loss_D_A = self.backward_D(self.netD_A, real_B, fake_B)
        loss_D_B = self.backward_D(self.netD_B, real_A, fake_A)
        self.optimizer_D.step()

        return loss_G, loss_D_A, loss_D_B


In [18]:
import os
import torch
import torch.nn as nn
from torchvision.utils import save_image
from torchvision import transforms
from torch.utils.data import DataLoader
from models.networks import ResnetGenerator, NLayerDiscriminator

# Define multiple dataset paths with respective batch sizes and sample sizes
datasets = {
    "Rain100L": {
        "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/input",
        "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/target",
        "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_2/",
        "batch_size": 8,  # Custom batch size per dataset
        "sample_size": 100  # Take only 100 samples
    },
    "DID-MDN-Heavy": {
        "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/rainy",
        "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/non_rainy",
        "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/",
        "batch_size": 6,
        "sample_size": 200  # Take 200 samples
    }
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Medium-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # },
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Light-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # }
}

# Define transformation with normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Define a function to denormalize images before saving
def denormalize(tensor):
    """Reverse normalization applied earlier to get back to [0, 1]"""
    return tensor * 0.5 + 0.5

# Initialize Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=128, ndf=128, device=device)

# Add Learning Rate Schedulers
scheduler_G = torch.optim.lr_scheduler.StepLR(model.optimizer_G, step_size=20, gamma=0.5)
scheduler_D = torch.optim.lr_scheduler.StepLR(model.optimizer_D, step_size=20, gamma=0.5)

# Training Configuration
num_epochs = 70

# Define a masked output directory
masked_output_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images"

# Loop Through Each Dataset and Train
for dataset_name, paths in datasets.items():
    print(f"\n🔹 Training on {dataset_name} dataset...")

    input_dir = paths["input"]
    target_dir = paths["target"]
    output_dir = paths["output"]
    batch_size = paths["batch_size"]
    sample_size = paths["sample_size"]

    os.makedirs(output_dir, exist_ok=True)

    # Load dataset with masked saving enabled
    train_loader = DataLoader(
        RainDatasetSSL(
            dataset_name=dataset_name,
            input_dir=input_dir,
            target_dir=target_dir,
            transform=transform,
            sample_size=sample_size,
            output_dir=masked_output_dir,  # Save masked images here
            save_masked=True
        ),
        batch_size=batch_size,
        shuffle=True
    )

    # Training Loop
    for epoch in range(num_epochs):
        model.netG_A.train()
        model.netG_B.train()
        epoch_loss_G, epoch_loss_D_A, epoch_loss_D_B = 0, 0, 0

        for i, (masked_A, real_A, real_B, img_name) in enumerate(train_loader):
          masked_A, real_A, real_B = masked_A.to(device), real_A.to(device), real_B.to(device)

          # Forward pass and optimization
          loss_G, loss_D_A, loss_D_B = model.optimize_parameters(masked_A, real_B)
          epoch_loss_G += loss_G.item()
          epoch_loss_D_A += loss_D_A.item()
          epoch_loss_D_B += loss_D_B.item()

          # Prevent runtime disconnection
          print(f"Epoch {epoch}/{num_epochs}, Batch {i}/{len(train_loader)}, Image: {img_name[0]}, "
                f"Loss_G: {loss_G:.4f}, Loss_D_A: {loss_D_A:.4f}, Loss_D_B: {loss_D_B:.4f}")

        # Step the scheduler
        scheduler_G.step()
        scheduler_D.step()

    # Save generated images at intervals
    if epoch == num_epochs - 1:
        model.netG_A.eval()  # Switch generator to evaluation mode
        with torch.no_grad():
            for i, (real_A, _, img_names) in enumerate(train_loader):
                real_A = real_A.to(device)
                fake_B = model.netG_A(real_A)  # Rainy -> Clean

                # Denormalize before saving
                fake_B = denormalize(fake_B)

                # Save each image in the batch
                for j in range(real_A.size(0)):
                    img_name = img_names[j]
                    save_path = os.path.join(output_dir, f"{dataset_name}_epoch_{epoch}_de_rained_{img_name}")
                    save_image(fake_B[j], save_path)

                    print(f"Saved: {save_path}")

print("Training on all datasets completed ✅")



Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



🔹 Training on Rain100L dataset...
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_24.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_95.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_98.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_73.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_55.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_76.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_11.png
Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/masked_images/masked_Rain100L_epoch_0_de_rained_87.png


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 2598 has 14.72 GiB memory in use. Of the allocated memory 12.99 GiB is allocated by PyTorch, and 1.61 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision.utils import save_image
from torchvision import transforms, models
from PIL import Image

# Define Autoencoder with Skip Connections
class AutoencoderWithSkipConnections(nn.Module):
    def __init__(self, input_nc):
        super(AutoencoderWithSkipConnections, self).__init__()
        self.encoder1 = nn.Sequential(nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.bottleneck = nn.Sequential(nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder1 = nn.Sequential(nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder2 = nn.Sequential(nn.ConvTranspose2d(256 + 256, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder3 = nn.Sequential(nn.ConvTranspose2d(128 + 128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder4 = nn.Sequential(nn.ConvTranspose2d(64 + 64, input_nc, kernel_size=4, stride=2, padding=1), nn.Tanh())

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        b = self.bottleneck(e3)
        d1 = self.decoder1(b)
        d2 = self.decoder2(torch.cat([d1, e3], dim=1))
        d3 = self.decoder3(torch.cat([d2, e2], dim=1))
        d4 = self.decoder4(torch.cat([d3, e1], dim=1))
        return d4

# Perceptual Loss using VGG19
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        vgg19 = models.vgg19(pretrained=True).features[:16].eval()
        for param in vgg19.parameters():
            param.requires_grad = False
        self.vgg19 = vgg19
        self.criterion = nn.MSELoss()

    def forward(self, x, y):
        x_features = self.vgg19(x)
        y_features = self.vgg19(y)
        return self.criterion(x_features, y_features)

# Initialize Autoencoder and Loss Functions
input_channels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = AutoencoderWithSkipConnections(input_channels).to(device)

mse_loss = nn.MSELoss()
perceptual_loss = PerceptualLoss().to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0002)

# Define the GAN-generated output directories
gan_output_dirs = {
    "results_6": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/Split-DID-MDN/results_6",
    "results_5": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/Split-DID-MDN/results_5",
    "results_4": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/Split-DID-MDN/results_4"
}

# Function to load images from a directory
def load_images_from_dir(directory):
    images, filenames = [], []
    for img_name in sorted(os.listdir(directory)):
        if img_name.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, img_name)
            img = Image.open(img_path).convert("RGB")
            img = transforms.ToTensor()(img)
            images.append(img)
            filenames.append(img_name)
    return torch.stack(images), filenames

# Process each directory separately
for dir_name, gan_output_path in gan_output_dirs.items():
    print(f"\n🔹 Processing GAN outputs from: {gan_output_path}")

    # Load images from current directory
    train_data, train_filenames = load_images_from_dir(gan_output_path)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

    # Train the Autoencoder
    epochs = 60
    final_epoch_reconstructions = {}

    for epoch in range(epochs):
        for i, images in enumerate(train_loader):
            images = images.to(device)
            reconstructed = autoencoder(images)
            mse_loss_value = mse_loss(reconstructed, images)
            perceptual_loss_value = perceptual_loss(reconstructed, images)
            total_loss = mse_loss_value + 0.01 * perceptual_loss_value

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            # Save final epoch reconstructions
            if epoch == epochs - 1:
                batch_filenames = train_filenames[i * 16:(i + 1) * 16]
                for j, filename in enumerate(batch_filenames):
                    final_epoch_reconstructions[filename] = reconstructed[j].unsqueeze(0)

        print(f"Epoch [{epoch+1}/{epochs}], MSE Loss: {mse_loss_value.item()}, Perceptual Loss: {perceptual_loss_value.item()}")

    # Create output directory for this specific dataset
    output_dir = f"/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/reconstructed_autoencoder_{dir_name}"
    os.makedirs(output_dir, exist_ok=True)

    # Save reconstructed images in their respective folders
    for filename, reconstructed_image in final_epoch_reconstructions.items():
        save_path = os.path.join(output_dir, f"{filename.split('.')[0]}_refined.png")
        save_image(reconstructed_image, save_path)

    print(f"✅ Finished processing {dir_name} - Saved to: {output_dir}")

print("\n🎯 Autoencoder Training and Refinement Completed for All GAN Output Directories!")


Epoch [1/60], MSE Loss: 0.20659932494163513, Perceptual Loss: 4.693155765533447
Epoch [2/60], MSE Loss: 0.13648764789104462, Perceptual Loss: 2.8803763389587402
Epoch [3/60], MSE Loss: 0.055457036942243576, Perceptual Loss: 3.4117679595947266
Epoch [4/60], MSE Loss: 0.03448586165904999, Perceptual Loss: 2.597916841506958
Epoch [5/60], MSE Loss: 0.019618110731244087, Perceptual Loss: 2.0173041820526123
Epoch [6/60], MSE Loss: 0.018136020749807358, Perceptual Loss: 1.9500073194503784
Epoch [7/60], MSE Loss: 0.017502862960100174, Perceptual Loss: 2.065246343612671
Epoch [8/60], MSE Loss: 0.012627768330276012, Perceptual Loss: 1.7568596601486206
Epoch [9/60], MSE Loss: 0.015493995510041714, Perceptual Loss: 1.8104727268218994
Epoch [10/60], MSE Loss: 0.009665400721132755, Perceptual Loss: 1.7112977504730225
Epoch [11/60], MSE Loss: 0.01640314981341362, Perceptual Loss: 1.7680519819259644
Epoch [12/60], MSE Loss: 0.009533348493278027, Perceptual Loss: 1.5512104034423828
Epoch [13/60], MSE L

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# If a GPU is available, you can also check its details
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")


Using device: cuda
GPU Name: Tesla T4
GPU Memory Allocated: 0.00 GB
GPU Memory Cached: 0.00 GB
